# Classical ML
### Notebook Deprecated
Deprecated, classical models don't seem to be working very well (because of lack of context, will fix this later).

In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud  
import sys
import sklearn
from sklearn.metrics import classification_report, make_scorer
from utils import *

In [2]:
DB_DIR = "../data"
DATA_NAME = "BC5CDR-chem"
DATA_DIR = path(DB_DIR, DATA_NAME)
TRAIN = "train.tsv"
DEV = "devel.tsv"
TEST = "test.tsv"

train_data = read_data(path(DATA_DIR, TRAIN))
dev_data = read_data(path(DATA_DIR, DEV))
test_data = read_data(path(DATA_DIR, TEST))

# LightGBM on statistical features

In [3]:
def w2f(sent, i):
    """naive feature engineering. Context is only +-1 word"""
    word = sent[i]
    features = {
        'word.len()' : len(word),
        'word.isdigit()' : word.isdigit(),
        'word.isupper()' : word.isupper(),
        'word.islower()' : word.islower(),
        'word.istitle()' : word.istitle(),
    }
    
    if i > 0:
        word1 = sent[i-1]
        features.update({
        'len(word-1)' : len(word1),
        'word-1.isdigit()' : word1.isdigit(),
        'word-1.isupper()' : word1.isupper(),
        'word-1.islower()' : word1.islower(),
        'word-1.istitle()' : word1.istitle(),
    })
    else:
        features.update({
        'len(word-1)' : -1,
        'word-1.isdigit()' : -1,
        'word-1.isupper()' : -1,
        'word-1.islower()' : -1,
        'word-1.istitle()' : -1,
    })
    
    if i < len(sent)-1:
        word1 = sent[i+1]
        features.update({
        'len(word+1)' : len(word1),
        'word+1.isdigit()' : word1.isdigit(),
        'word+1.isupper()' : word1.isupper(),
        'word+1.islower()' : word1.islower(),
        'word+1.istitle()' : word1.istitle(),
    })
    else:
        features.update({
        'len(word+1)' : -1,
        'word+1.isdigit()' : -1,
        'word+1.isupper()' : -1,
        'word+1.islower()' : -1,
        'word+1.istitle()' : -1,
    })
    return features    

In [4]:
def prepare_data(data):
    X, y = [], []
    for instance in data:
        labels, text = instance[0].split(), instance[1].split()
        X += [w2f(text, i) for i in range(len(text))]
        y += labels
    X = pd.DataFrame(X)
    assert X.shape[0] == len(y), "Failure"
    return pd.DataFrame(X), y    

In [5]:
X_train, y_train = prepare_data(train_data)
X_dev, y_dev = prepare_data(dev_data)
X_test, y_test = prepare_data(test_data)

In [6]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(y_train)
print(f"Encoding categorical labels: {le.classes_}")

y_train = le.transform(y_train)
y_dev = le.transform(y_dev)
y_test = le.transform(y_test)

for col in X_train.columns:
    X_train[col] = X_train[col].astype(int)
    
for col in X_dev.columns:
    X_dev[col] = X_dev[col].astype(int)
    
for col in X_test.columns:
    X_test[col] = X_test[col].astype(int)    

Encoding categorical labels: ['B' 'I' 'O']


In [7]:
import lightgbm as lgb

params = {"objective": "multiclass",
          "num_class": 3,
          "boosting_type": "gbdt",
          "learning_rate": 0.1,
          'max_depth': 5,
          "num_leaves": 5,
          "max_bin": 32,
          "feature_fraction": 0.7,
          "verbosity": 0,
          "drop_rate": 0.1,
          "max_drop": 50,
          "min_child_samples": 10,
          "min_sum_hessian_in_leaf": 10,
          "bagging_fraction": 0.7,
          "bagging_freq": 5,
          "random_seed" : 123123123,
        }

num_boost_round = int(1e5)
early_stopping_rounds = 500
verbose_eval = 300

train_dataset = lgb.Dataset(X_train, y_train)
dev_dataset = lgb.Dataset(X_dev, y_dev)
test_dataset = lgb.Dataset(X_test, y_test)

In [8]:
model = lgb.train(
            params, 
            train_dataset, 
            num_boost_round=num_boost_round,
            valid_sets=[train_dataset, dev_dataset],
            verbose_eval=verbose_eval,
            early_stopping_rounds=early_stopping_rounds
        )

Training until validation scores don't improve for 500 rounds
[300]	training's multi_logloss: 0.177375	valid_1's multi_logloss: 0.182315
[600]	training's multi_logloss: 0.171879	valid_1's multi_logloss: 0.179869
[900]	training's multi_logloss: 0.168674	valid_1's multi_logloss: 0.178618
[1200]	training's multi_logloss: 0.16655	valid_1's multi_logloss: 0.178228
[1500]	training's multi_logloss: 0.165163	valid_1's multi_logloss: 0.178041
[1800]	training's multi_logloss: 0.16405	valid_1's multi_logloss: 0.177898
[2100]	training's multi_logloss: 0.163098	valid_1's multi_logloss: 0.177749
[2400]	training's multi_logloss: 0.162302	valid_1's multi_logloss: 0.177808
Early stopping, best iteration is:
[2144]	training's multi_logloss: 0.162969	valid_1's multi_logloss: 0.177692


In [9]:
y_dev_pred = model.predict(X_dev)
y_test_pred = model.predict(X_test)

In [10]:
report = classification_report(y_dev, y_dev_pred.argmax(axis=1))
print(report)

              precision    recall  f1-score   support

           0       0.57      0.12      0.19      5347
           1       0.66      0.14      0.24      1748
           2       0.95      0.99      0.97    110358

    accuracy                           0.94    117453
   macro avg       0.72      0.42      0.47    117453
weighted avg       0.93      0.94      0.92    117453



In [18]:
def infer(sentence, model):
    sentence = sentence.split()
    X = [w2f(sentence, i) for i in range(len(sentence))]
    X = pd.DataFrame(X)
    for col in X.columns:
        X[col] = X[col].astype(int)
    return model.predict(X)

In [26]:
sentence = "She suffers diabetes."
out = infer(sentence, model).argmax(axis=1)

# Conditional Random Fields

In [ ]:
def w2f(sent, i):
    word = sent[i]
    features = {
        'bias': 1.0,
        'word.lower()' : word.lower(),
        'word[-3:]' : word[-3:],
        'word[-2:]' : word[-2:],
        'word.isupper()' : word.isupper(),
        'word.islower()' : word.islower(),
        'word.istitle()' : word.istitle(),
    }
    
    if i > 0:
        word1 = sent[i-1]
        features.update({
        'word-1.lower()' : word1.lower(),
        'word-1.isupper()' : word1.isupper(),
        'word-1.islower()' : word1.islower(),
        'word-1.istitle()' : word1.istitle(),
    })
    else:
        features['BOS'] = True
    
    if i < len(sent)-1:
        word1 = sent[i+1]
        features.update({
        'word+1.lower()' : word1.lower(),
        'word+1.isupper()' : word1.isupper(),
        'word+1.islower()' : word1.islower(),
        'word+1.istitle()' : word1.istitle(),
    })
    else:
        features['EOS'] = True
    return features    

In [ ]:
def prepare_data(data):
    X, y = [], []
    for instance in data:
        labels, text = instance[0].split(), instance[1].split()
        X.append([w2f(text, i) for i in range(len(text))])
        y.append(labels)
    return X, y    

In [ ]:
X_train, y_train = prepare_data(train_data)
X_dev, y_dev = prepare_data(dev_data)
X_test, y_test = prepare_data(test_data)

In [ ]:
from sklearn_crfsuite import CRF, scorers, metrics
from sklearn_crfsuite.metrics import flat_classification_report

crf = CRF(algorithm='lbfgs',
          c1=10,
          c2=0.1,
          max_iterations=1000,
          all_possible_transitions=False,
         )

In [ ]:
model = crf.fit(X_train, y_train)

In [ ]:
y_dev_pred = model.predict(X_dev)
y_test_pred = model.predict(X_test)

In [ ]:
report = flat_classification_report(y_dev, y_dev_pred)
print(report)

In [ ]:
report = flat_classification_report(y_test, y_test_pred)
print(report)